In [1]:
from langchain.vectorstores.cassandra import Cassandra
from langchain.indexes.vectorstore import VectorStoreIndexWrapper
from langchain.llms import OpenAI
from langchain.embeddings import OpenAIEmbeddings

from datasets import load_dataset

# engine powering Astra DB integreation in Langchain
# DB connection initilization
import cassio

In [2]:
!pip install PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 9.0 MB/s eta 0:00:00


In [3]:
from PyPDF2 import PdfReader

In [4]:
!pip install python-dotenv

  Using cached python_dotenv-1.0.1-py3-none-any.whl.metadata (23 kB)
Using cached python_dotenv-1.0.1-py3-none-any.whl (19 kB)


In [6]:
from dotenv import load_dotenv

load_dotenv() 

True

In [ ]:
!pip install --upgrade astrapy